## Kaggle ML2
## Matteo A. D'Alessandro, Carlo A. Patti

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
os.listdir()

['baseline_rf.ipynb', 'EDA.ipynb']

In [5]:
df = pd.read_csv('../data/train.csv').drop('Id', axis=1)

In [6]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2881,130,22,210,54,1020,250,221,88,342,...,0,0,0,0,0,0,0,0,0,1
1,3005,351,14,242,-16,1371,194,215,159,842,...,0,0,0,0,0,0,0,0,0,1
2,3226,63,14,618,2,1092,232,210,107,2018,...,0,0,0,0,0,0,0,0,0,1
3,3298,317,8,661,60,752,198,233,174,1248,...,0,0,0,0,0,0,0,0,0,1
4,3080,35,6,175,26,3705,219,227,144,2673,...,0,0,0,0,0,0,0,0,0,1


Converting horizontal and vertical distances to hydrology to the Euclidean one

In [11]:
df['Distance_To_Hydrology'] = np.sqrt(df['Horizontal_Distance_To_Hydrology']**2 + df['Vertical_Distance_To_Hydrology']**2)
df.drop(['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology'], axis=1, inplace=True)

# Climate and Geologic Zones

## First Digit: Climatic Zone

- **1**: Lower Montane Dry
- **2**: Lower Montane
- **3**: Montane Dry
- **4**: Montane
- **5**: Montane Dry and Montane
- **6**: Montane and Subalpine
- **7**: Subalpine
- **8**: Alpine

## Second Digit: Geologic Zones

- **1**: Alluvium
- **2**: Glacial
- **3**: Shale
- **4**: Sandstone
- **5**: Mixed Sedimentary
- **6**: Unspecified in the USFS ELU Survey
- **7**: Igneous and Metamorphic
- **8**: Volcanic

In [12]:
soil_cols = [col for col in df.columns if col.startswith('Soil')]
df['soil_type'] = df[soil_cols].idxmax(axis=1).str.replace('Soil_Type','').astype(int)
 

In [13]:
elu_data_raw = pd.read_csv('..//data//num_to_elu.txt', sep=":", header=None)

In [14]:
elu_data_raw['ELU'] = elu_data_raw[1].str[5:9]

elu_data_raw['climatic_zone'] = elu_data_raw['ELU'].str[0]
elu_data_raw['geologic_zone'] = elu_data_raw['ELU'].str[1]

elu_data_raw.drop([1, 'ELU'], axis=1, inplace=True)
elu_data_raw.columns = ['soil_type', 'climatic_zone', 'geologic_zone']

In [16]:
df = df.merge(elu_data_raw, on='soil_type', how='left')
df.drop(['soil_type'], axis=1, inplace=True)

In [18]:
elu_data_raw.head()

,soil_type,climatic_zone,geologic_zone
0,1,2,7
1,2,2,7
2,3,2,7
3,4,2,7
4,5,2,7


In [10]:
y = df['Cover_Type']
X = df.drop(['Cover_Type', 'Id'], axis=1)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

0.8644179894179894
              precision    recall  f1-score   support

           1       0.77      0.79      0.78       427
           2       0.83      0.69      0.75       464
           3       0.85      0.83      0.84       400
           4       0.93      0.98      0.95       436
           5       0.89      0.95      0.92       452
           6       0.84      0.86      0.85       415
           7       0.94      0.96      0.95       430

    accuracy                           0.86      3024
   macro avg       0.86      0.87      0.86      3024
weighted avg       0.86      0.86      0.86      3024

